In [1]:
import csv
import sqlite3
# from flask import jsonify
import json
# import pandas as pd


In [13]:
def connect(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    return conn, cursor


In [3]:
def insert_some_data():
    try:
        with open('clustered_articles_small.csv', 'r') as data:
            reader = csv.DictReader(data)
            to_db = [(i['article_id'], i["Keywords"], i['Title'], i["Text"], i["Bias"], i['Cluster Tags']) for i in reader]

        cursor.executemany("INSERT INTO Articles (article_id, Title, Text, Bias, Keywords, [Cluster Tags]) VALUES (?, ?, ?, ?, ?, ?);", to_db)
    #     con.commit()
    #     cur.close()
    except sqlite3.Error as error:
        print("Error: ", error)
    # finally:
    #     con.close()
    #     print("sqlite connection closed")

In [32]:
def get_histogram_data():
    # cluster_tags = [
    #     "Political and Social Issues",
    #     "Community Involvement",
    #     "Names, Organizations, and Various Terms",
    #     "Locations",
    #     "Misc. Adjectives",
    #     "Legal and Law Enforcement",
    #     "Science and Medicine",
    #     "Noise/Nonsensical Phrases",
    #     "Media and Entertainment",
    #     "Various Phrases/Verbs",
    #     "Nature and Wildlife",
    #     "Food",
    #     "Finance and Economics",
    #     "Objects and Accessories",
    # ]
    conn = sqlite3.connect('first_2000.db')
    conn = conn.cursor()
    conn.execute(
        """
        SELECT cluster_id, cluster_name from clusters;"""
    )
    cluster_tags = conn.fetchall()
    cluster_tags = [tag[1] for tag in cluster_tags[1:]]
    
    # this one uses small.db, created this from shell
    '''
    sqlite3 small.db
    .mode csv articles
    .import clustered_articles_small.csv articles  # this is the first 2000 rows of large csv
    .tables  # prints 'articles'
    .schema  # also gives more info
    .quit
    '''
    conn.execute("Select bias, count(*) from articles where [Cluster Tags] like '%Nature And WildLife%' group by bias")
    data = conn.fetchall()
    print(data)

    # conn = sqlite3.connect('small.db')
    # conn = conn.cursor()


    # histogram_data = []
    # for tag in cluster_tags:
    #     print("current tag", tag)
    #     like_tag = "%" + tag + "%"
    #     conn.execute("""
    #         SELECT bias, count(*) AS article_count FROM articles 
    #         WHERE [Cluster Tags] LIKE ?
    #         """, (like_tag,))
    #     cluster_data = conn.fetchall()
    #     print(cluster_data)


        # histogram_data.append({
        #     "tag": tag,
        #     "data": cluster_data
        # })
    # conn.close()

    return json.dumps(data)

In [33]:
get_histogram_data()

[('0', 12), ('1', 11), ('2', 30), ('3', 1), ('4', 18)]


'[["0", 12], ["1", 11], ["2", 30], ["3", 1], ["4", 18]]'